# **Naive Bayes Explained with a Simple Spam Filter Example**

### **Libraries used**

In [1]:
import re
import glob
import random
import numpy as np
from statistics import mean
from math import log, exp
from collections import defaultdict
from sklearn.metrics import confusion_matrix
from typing import List, Set, Dict, Iterable, Tuple, Literal
from scratch.metrics import precision, recall, f1_score
from scratch.data_preprocessing import Message, split_data, print_classification_report


### **Introduction**

**Bayes' Theorem and Its Role in Machine Learning**

Bayes' Theorem is a fundamental concept in probability theory that provides a mathematical framework for updating beliefs in light of new evidence. Named after the 18th-century statistician Thomas Bayes, the theorem describes how to compute the probability of an event $A$ given that another event $B$ has occurred. Formally, it is expressed as:

$$
P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}
$$

Where:

- $P(A|B)$ is the **posterior probability**: the updated probability of $A$ given that $B$ has occurred,
- $P(B|A)$ is the **likelihood**: the probability of observing $B$ assuming $A$ is true,
- $P(A)$ is the **prior probability**: our initial belief about $A$ before seeing $B$,
- $P(B)$ is the **marginal probability**: the overall probability of observing $B$, regardless of $A$.

In the next sections, we will explore the logic behind this formula.
<br>

**What You’ll Learn**

In this notebook, we’ll walk through the implementation of a simple spam filter using the Naive Bayes model. Through this example, you’ll gain a deeper understanding of how Bayes’ Theorem works in practice and how probabilistic models can be applied to solve real-world classification problems.

### **Theoretical Framework**

**Bayes' Theorem in Machine Learning**

In machine learning, Bayes' Theorem forms the foundation of a group of algorithms called **Bayesian classifiers**. One of the most popular and practical of these is the **Naive Bayes classifier**.

The term *naive* refers to a simplifying assumption: it assumes that all features (e.g., words in an email) are conditionally independent given the class (e.g., spam or not spam). While this assumption is rarely true in real-world data, the model often works surprisingly well — especially in applications involving high-dimensional data like text.

Naive Bayes is widely used in tasks such as:
- Spam filtering,
- Sentiment analysis,
- Document classification.

Its strengths include simplicity, interpretability, speed, and robustness — even with relatively small datasets.

Let's see the maths behind the scenes.

We start with the definition of conditional probability:

$$
\tag{1}
P(B|A) = \frac{P(A \cap B)}{P(A)}
$$

Equation (1) states that the probability of event $B$ occurring given event $A$ has occurred is the probability of both $A$ and $B$ happening together divided by the probability of $A$.

Similarly, the conditional probability of $A$ given $B$ is:

$$
\tag{2}
P(A|B) = \frac{P(A \cap B)}{P(B)}
$$

Note that the joint probability $P(A \cap B)$ appears in both equations (1) and (2). Rearranging equation (1) gives us:

$$
\tag{3}
P(A \cap B) = P(B|A) \cdot P(A)
$$

Substituting equation (3) into equation (2) yields Bayes Theorem:

$$
\tag{4}
P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}
$$

This formula (4) allows us to **invert** the conditional probability: it expresses the probability of $A$ given $B$ in terms of the probability of $B$ given $A$, the prior probability of $A$, and the overall probability of $B$.

Bayes' Theorem provides a powerful and elegant mathematical framework for reasoning about uncertainty and updating beliefs based on new evidence. Its ability to reverse conditional probabilities makes it an essential tool in machine learning, particularly for classification tasks.

The Naive Bayes classifier builds on this theorem with a simplifying assumption of feature independence, which makes it computationally efficient and easy to implement. Despite its simplicity, it remains highly effective in many practical scenarios, especially in text-related problems such as spam filtering, where it leverages probabilistic reasoning to make informed predictions.

Understanding this theoretical framework is crucial before diving into the implementation details and real-world applications.

With the theory in place, let’s dive into a practical example to see it in action

### **Practical exercise - A Spam Filter**

To see how Bayes Theorem works in practice, let’s apply it to a simple spam filter scenario. Imagine we want to estimate the probability that an email is spam, given that it contains a specific word — also called a token — such as “free” or “discount”.

In this context, we treat:
- $A$ as the event “the email contains the token”, and
- $B$ as the event “the email is spam”.

Using Bayes’ Theorem, we can compute the probability that a message is spam given it contains the token, as follows:

$$
P(\text{spam}|\text{token}) = \frac{P(\text{token}|\text{spam}) \cdot P(\text{spam})}{P(\text{token})}
$$

Let’s break this down:
- $P(\text{spam}|\text{token})$ is the posterior: the probability that the email is spam, given it contains the token.
- $P(\text{token}|\text{spam})$ is the likelihood: how often this token appears in known spam emails.
- $P(\text{spam})$ is the prior: the overall probability that any email is spam, regardless of content.
- $P(\text{token})$ is the evidence: how frequently the token appears in all emails, spam or not.

This equation gives us a systematic way to update our belief — how likely an email is spam — based on the presence of specific words. It forms the core of how probabilistic spam filters make decisions based on message content.

The denominator, $P(\text{token})$, represents the total probability of seeing the token in any email — spam or not.

$$
P(\text{token}) = P(\text{token}|\text{spam}) \cdot P(\text{spam}) + P(\text{token}|\neg \text{spam}) \cdot P(\neg \text{spam})
$$

This expression accounts for two scenarios:
1. The token appears in a spam email.
2. The token appears in a non-spam (ham) email.

Now that we understand each component, we can bring them all together in a complete formula. This gives us a practical expression to calculate the probability that an email is spam, given that it contains a specific token:

$$
\tag{1}
P(\text{spam}|\text{token}) = \frac{P(\text{token}|\text{spam}) \cdot P(\text{spam})}{P(\text{token}|\text{spam}) \cdot P(\text{spam}) + P(\text{token}|\neg \text{spam}) \cdot P(\neg \text{spam})}
$$

Let’s assume we have no reason to believe that spam or non-spam messages are more frequent — in other words:

$$
\tag{2}
P(\text{spam}) = P(\neg \text{spam}) = 0.5
$$

Substituting equation (2) into equation (1):

$$
\tag{3}
P(\text{spam}|\text{token}) = \frac{P(\text{token}|\text{spam}) \cdot 0.5}{P(\text{token}|\text{spam}) \cdot 0.5 + P(\text{token}|\neg \text{spam}) \cdot 0.5}
$$

Now factor out the common prior:

$$
P(\text{spam}|\text{token}) = \frac{0.5 \cdot P(\text{token}|\text{spam})}{0.5 \cdot \left[P(\text{token}|\text{spam}) + P(\text{token}|\neg \text{spam})\right]}
$$

Cancel the 0.5 from numerator and denominator:

$$
\tag{4}
P(\text{spam}|\text{token}) = \frac{P(\text{token}|\text{spam})}{P(\text{token}|\text{spam}) + P(\text{token}|\neg \text{spam})}
$$

This simplified expression (4) allows us to estimate the probability that an email is spam just based on the relative likelihoods of the token appearing in spam vs. non-spam messages — assuming equal priors.



**Extending to Multiple Tokens**

To handle an entire email, we consider the presence of multiple tokens (words). Assuming conditional independence between tokens — the core assumption of Naive Bayes — we can multiply Equation (4) for each token in the message:
$$
P(\text{spam}|\text{tokens}) =
\frac{
\prod_{i=1}^n P(\text{token}_i|\text{spam})
}
{
\prod_{i=1}^n P(\text{token}_i|\text{spam}) +
\prod_{i=1}^n P(\text{token}_i|\neg \text{spam})
}
$$

This formula estimates the probability that a message is spam given all its tokens. Each term in the product reflects how strongly a word supports the “spam” or “not spam” hypothesis.


**Handling Unseen Tokens**

When a token has never appeared in the training data for a class (e.g., spam), we get:

$$
P(\text{token}_i|\text{spam}) = 0
$$

This would nullify the entire product. To avoid this, we apply a pseudocount smoothing:

$$
P(\text{token}_i|\text{spam}) = \frac{\text{count}(\text{token}_i|\text{spam}) + k}{\text{total tokens in spam} + 2k}
$$

**Avoiding Underflow with Log Probabilities**

When multiplying many small probabilities, the result can become extremely close to zero, causing numerical underflow — a problem where the computer cannot accurately represent very small numbers.

To avoid this, we perform calculations in the logarithmic domain, using the property:

$$
\log(ab) = \log(a) + \log(b)
$$

This means that instead of multiplying probabilities directly, we sum their logarithms, which is numerically more stable.

Once we have the sum of the logs, we can convert back to the original probability by exponentiating:

$$
ab = \exp\big(\log(ab)\big)
$$

Now we start with the example.

**Getting data**

We will be using the corpus data from: https://spamassassin.apache.org/old/publiccorpus/, specifically the subjects.

In [2]:
path = "datasets/spam_data/*/*"
data: List[Message] = []
for filename in glob.glob(path):
    is_spam = "ham" not in filename
    with open(filename, errors='ignore') as email_file:
        for line in email_file:
            if line.startswith("Subject:"):
                subject = line.lstrip("Subject: ")
                data.append(Message(subject, is_spam))
                break

Now we will code a function for tokenize

In [3]:
def tokenize(text: str) -> Set[str]:
    """Reduce a text into a set of its words."""
    text = text.lower()
    all_words = re.findall("[a-z0-9]+", text)
    return set(all_words)

tokenize_example = tokenize('this function just keep a single token lets repeat this and function')
print(tokenize_example)

{'just', 'function', 'a', 'single', 'token', 'this', 'keep', 'lets', 'repeat', 'and'}


And a class for our model

In [4]:
class NaiveBayesSpamClassifier:
    """
    A model to predict if a message is spam where
    k = smothing value
    """
    def __init__(self, k: float = 0.5) -> None:
        self.k = k
        self.tokens = set()
        self.token_spam_count: Dict[str, int] = defaultdict(int)
        self.token_ham_count: Dict[str, int] = defaultdict(int)
        self.spam_messages = 0
        self.ham_messages = 0
    
    def fit(self, messages: Iterable[Message]) -> None:
        """
        Store the following:
        spam_messages = count of spam messages
        ham_messages = count of not spam messages
        token_spam_count = count(token|spam) just one count per token per mail
        token_ham_count = count(token|ham) just one count per token per mail
        """
        for message in messages:
            if message.is_spam:
                self.spam_messages += 1
            elif not message.is_spam:
                self.ham_messages += 1
        
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_count[token] += 1
                elif not message.is_spam:
                    self.token_ham_count[token] += 1
    
    def _probabilities(self, token: str) -> Tuple[float, float, float, float]:
        """
        Computes the probabilities listed below:
        P(token|spam)
        P(¬token|spam)
        P(token|ham)
        P(¬token|ham)
        """
        p_token_given_spam = (self.k + self.token_spam_count[token])/ ((2*self.k) + (self.spam_messages))
        p_not_token_given_spam = 1 - p_token_given_spam
        p_token_given_ham = (self.k + self.token_ham_count[token])/ ((2*self.k) + (self.ham_messages))
        p_not_token_given_ham = 1 - p_token_given_ham

        return p_token_given_spam, p_not_token_given_spam, p_token_given_ham, p_not_token_given_ham
    
    def predict(self, text: str) -> float:
        """
        Return the probability of spam given the tokens.
        """
        text_token = tokenize(text)
        log_prob_token_given_spam = 0
        log_prob_token_given_ham = 0

        # Iterate through each token and add the log of the probability
        for token in self.tokens:
            p_token_given_spam, p_not_token_given_spam, p_token_given_ham, p_not_token_given_ham = self._probabilities(token)
            
            if token in text_token:
                log_prob_token_given_spam += log(p_token_given_spam)
                log_prob_token_given_ham += log(p_token_given_ham)
            
            elif token not in text_token:
                log_prob_token_given_spam += log(p_not_token_given_spam)
                log_prob_token_given_ham += log(p_not_token_given_ham)
            
        p_tokens_given_spam = exp(log_prob_token_given_spam)
        p_tokens_given_ham = exp(log_prob_token_given_ham)

        return p_tokens_given_spam/(p_tokens_given_spam + p_tokens_given_ham)
    
    def _cm(self, test_messages: List[Message], threshold: float = 0.5) -> Dict[str, float]:
        """ Get the confusion matrix of each label
            Cols - Predicted
            Rows - Actual
        """
        real_labels = [message.is_spam for message in test_messages]
        predicted_labels = [self.predict(message.text)>threshold for message in test_messages]
        labels = sorted(list(set(real_labels + predicted_labels)), reverse=True)
        cm = confusion_matrix(real_labels, predicted_labels, labels=labels)
        
        cm_detailed = defaultdict(dict)
        for label_index in range(len(labels)):
            tp = 0
            tn = 0
            fp = 0
            fn = 0
            for row in range(len(labels)):
                for col in range(len(labels)):
                    if row==label_index and col==label_index:
                        tp += int(cm[row][col])
                    elif row==label_index and col!=label_index:
                        fn += int(cm[row][col])
                    elif row!=label_index and col==label_index:
                        fp += int(cm[row][col])
                    elif row!=label_index and col!=label_index:
                        tn += int(cm[row][col])

            cm_detailed[labels[label_index]]['tp'] = tp
            cm_detailed[labels[label_index]]['tn'] = tn
            cm_detailed[labels[label_index]]['fp'] = fp
            cm_detailed[labels[label_index]]['fn'] = fn

        return labels, cm, cm_detailed
    
    def metrics(self ,test_dataset: List[Message], 
                threshold: float = 0.5 ,
                kind: Literal['micro','macro'] = 'micro') -> Dict[str, float]:
        labels, cm, cm_detailed = self._cm(test_dataset, threshold = threshold)

        if len(labels) == 2:
            tp = cm_detailed[labels[0]]['tp']
            fp = cm_detailed[labels[0]]['fp']
            fn = cm_detailed[labels[0]]['fn']
            tn = cm_detailed[labels[0]]['tn']
            _accuracy = float(np.trace(cm)/np.sum(cm))
            _precision = precision(tp, fp)
            _recall = recall(tp, fn)
            _f1_score = f1_score(tp, tn, fp, fn)
            
            return {
                "labels": labels,
                "confusion_matrix": cm,
                "confusion_matrix_detailes": cm_detailed,
                "accuracy": _accuracy, 
                "precision": _precision, 
                "recall": _recall,
                "f1_score": _f1_score
            }

        elif len(labels) > 2:
            _accuracy = float(np.trace(cm)/np.sum(cm))
            if kind == 'micro':
                tp = sum([cm_detailed[label]['tp'] for label in labels])
                fp = sum([cm_detailed[label]['fp'] for label in labels])
                fn = sum([cm_detailed[label]['fn'] for label in labels])
                tn = sum([cm_detailed[label]['tn'] for label in labels])
                _precision = precision(tp, fp)
                _recall = recall(tp, fn)
                _f1_score = f1_score(tp, tn, fp, fn)

            elif kind == 'macro':
                _precision = mean(
                    [
                        precision(
                            cm_detailed[label]['tp'],
                            cm_detailed[label]['fp']
                        )
                        for label in labels
                    ]
                
                )

                _recall = mean(
                    [
                        recall(
                            cm_detailed[label]['tp'],
                            cm_detailed[label]['fn']
                        )
                        for label in labels
                    ]
                
                )

                _f1_score = mean(
                    [
                        f1_score(
                            cm_detailed[label]['tp'],
                            cm_detailed[label]['tn'],
                            cm_detailed[label]['fp'],
                            cm_detailed[label]['fn']
                        )
                        for label in labels
                    ]
                )

            else:
                raise AssertionError('Not a valid kind of metrics, use kind = "micro" or kind = "macro"')
            
            return {
                "labels": labels,
                "confusion_matrix": cm,
                "confusion_matrix_detailes": cm_detailed,
                "accuracy": _accuracy, 
                "precision": _precision, 
                "recall": _recall,
                "f1_score": _f1_score
            }

Training the model and making some predictions

In [8]:

random.seed(0)
train_messages, test_messages = split_data(data, .75)
model = NaiveBayesSpamClassifier()
model.fit(train_messages)
metrics = model.metrics(test_messages)
print_classification_report(metrics)


=== Classification Report ===
Labels: [True, False]

Confusion Matrix:
  85  54
  22  664

Detailed Confusion Metrics per Class:
  1            -> TP: 85, TN: 664, FP: 22, FN: 54
  0            -> TP: 664, TN: 85, FP: 54, FN: 22

Overall Metrics:
  Accuracy : 0.9079
  Precision: 0.7944
  Recall   : 0.6115
  F1 Score : 0.6911


Just be aware, what we got of this prediction is the $P(spam|tokens)$ so you must defin a threshold for considering a prediction like a spam

In [17]:
# Make 4 predictions
predictions = [(f"{message.text[:10]}...", message.is_spam, model.predict(message.text)) for message in test_messages[:4]]

for subject, real, predicted in predictions:
    print(f"Subject: {subject}")
    print(f"Real Is Spam: {real}")
    print(f"Predicted Is Spam: {predicted}")
    print(f"Predicted Is Spam Labeled: {predicted>0.50}\n")

Subject: Re: Signer...
Real Is Spam: False
Predicted Is Spam: 5.124896331778981e-07
Predicted Is Spam Labeled: False

Subject: Patch to c...
Real Is Spam: False
Predicted Is Spam: 0.01862463789721551
Predicted Is Spam Labeled: False

Subject: RE: [Razor...
Real Is Spam: False
Predicted Is Spam: 5.983072599443672e-07
Predicted Is Spam Labeled: False

Subject: Fw: NORTON...
Real Is Spam: True
Predicted Is Spam: 0.9999995774247706
Predicted Is Spam Labeled: True



### **Conclusion**

Bayes Theorem helps us figure out the probability of something based on new information. In spam filtering, it lets us estimate how likely an email is spam if it contains certain words.

The Naive Bayes model assumes that each word in an email is independent from the others, which makes calculations much easier and faster. Even though this assumption is simple, it works surprisingly well in practice.

Thanks to this method, we can build a spam filter that uses math to make smart guesses — and it’s both fast and easy to implement.